In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
import datetime as dt

# Data Viz 
import seaborn as sns
import matplotlib.pyplot as plt


# Similarity calculation
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix

# settings
pd.options.display.max_rows = 100
pd.options.display.max_columns = None

# Math
import math

# Remove warnings
import warnings
warnings.filterwarnings("ignore")

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/santander-product-recommendation/test_ver2.csv.zip
/kaggle/input/santander-product-recommendation/sample_submission.csv.zip
/kaggle/input/santander-product-recommendation/train_ver2.csv.zip


#### 1. Data Import and Profiling

In [2]:
# import the tables
train = pd.read_csv(filepath_or_buffer='/kaggle/input/santander-product-recommendation/train_ver2.csv.zip')
#test = pd.read_csv(filepath_or_buffer='/kaggle/input/santander-product-recommendation/test_ver2.csv.zip')

In [3]:
## Data Profiling
train.info()
train.describe(include = 'all').T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13647309 entries, 0 to 13647308
Data columns (total 48 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   fecha_dato             object 
 1   ncodpers               int64  
 2   ind_empleado           object 
 3   pais_residencia        object 
 4   sexo                   object 
 5   age                    object 
 6   fecha_alta             object 
 7   ind_nuevo              float64
 8   antiguedad             object 
 9   indrel                 float64
 10  ult_fec_cli_1t         object 
 11  indrel_1mes            object 
 12  tiprel_1mes            object 
 13  indresi                object 
 14  indext                 object 
 15  conyuemp               object 
 16  canal_entrada          object 
 17  indfall                object 
 18  tipodom                float64
 19  cod_prov               float64
 20  nomprov                object 
 21  ind_actividad_cliente  float64
 22  renta           

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
fecha_dato,13647309,17,2016-05-28,931453,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ncodpers,13647309.0,NaN,NaN,NaN,834904.211501,431565.025784,15889.0,452813.0,931893.0,1199286.0,1553689.0
ind_empleado,13619575,5,N,13610977,NaN,NaN,NaN,NaN,NaN,NaN,NaN
pais_residencia,13619575,118,ES,13553710,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sexo,13619505,2,V,7424252,NaN,NaN,NaN,NaN,NaN,NaN,NaN
age,13647309.0,235.0,23.0,542682.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fecha_alta,13619575,6756,2014-07-28,57389,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ind_nuevo,13619575.0,NaN,NaN,NaN,0.059562,0.236673,0.0,0.0,0.0,0.0,1.0
antiguedad,13647309.0,507.0,0.0,134335.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
indrel,13619575.0,NaN,NaN,NaN,1.178399,4.177469,1.0,1.0,1.0,1.0,99.0


In [4]:
# Check missing values
train.isnull().sum()/train.shape[0] * 100

fecha_dato                0.000000
ncodpers                  0.000000
ind_empleado              0.203220
pais_residencia           0.203220
sexo                      0.203732
age                       0.000000
fecha_alta                0.203220
ind_nuevo                 0.203220
antiguedad                0.000000
indrel                    0.203220
ult_fec_cli_1t           99.818330
indrel_1mes               1.097513
tiprel_1mes               1.097513
indresi                   0.203220
indext                    0.203220
conyuemp                 99.986752
canal_entrada             1.363829
indfall                   0.203220
tipodom                   0.203227
cod_prov                  0.685784
nomprov                   0.685784
ind_actividad_cliente     0.203220
renta                    20.475648
segmento                  1.387585
ind_ahor_fin_ult1         0.000000
ind_aval_fin_ult1         0.000000
ind_cco_fin_ult1          0.000000
ind_cder_fin_ult1         0.000000
ind_cno_fin_ult1    

In [5]:
## Inspect the data sample
train.sample(5)

,fecha_dato,ncodpers,ind_empleado,pais_residencia,sexo,age,fecha_alta,ind_nuevo,antiguedad,indrel,ult_fec_cli_1t,indrel_1mes,tiprel_1mes,indresi,indext,conyuemp,canal_entrada,indfall,tipodom,cod_prov,nomprov,ind_actividad_cliente,renta,segmento,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
13434452,2016-05-28,1043604,N,ES,V,23,2012-08-06,0.0,45,1.0,NaN,1.0,I,S,N,NaN,KHD,N,1.0,33.0,ASTURIAS,0.0,NaN,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
4174599,2015-07-28,281510,N,ES,V,41,2001-10-11,0.0,165,1.0,NaN,1.0,A,S,N,NaN,KFC,N,1.0,50.0,ZARAGOZA,1.0,94580.28,02 - PARTICULARES,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
12112958,2016-04-28,173615,N,ES,H,75,2000-04-24,0.0,192,1.0,NaN,1,I,S,N,NaN,KFC,N,1.0,47.0,VALLADOLID,0.0,106095.57,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
9969944,2016-02-28,1311316,N,ES,H,23,2014-09-18,0.0,17,1.0,NaN,1.0,A,S,N,NaN,KHE,N,1.0,20.0,GIPUZKOA,1.0,NaN,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
1063557,2015-02-28,1287352,N,ES,H,22,2014-08-01,0.0,12,1.0,NaN,1.0,I,S,N,NaN,KHE,N,1.0,3.0,ALICANTE,0.0,70167.27,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


In [6]:
# 1) Feature Name Transformation
col_names = {"ncodpers":"cust_id", "ind_empleado":"emp_index","pais_residencia":"residence",
            "sexo":"sex","fecha_alta":"first_date","ind_nuevo":"new_cust","antiguedad":"seniority",
            "indrel":"is_primary","ult_fec_cli_1t":"last_primary_date","indrel_1mes":"cust_type",
            "tiprel_1mes":"cust_rel_type","indresi":"residence_index","indext":"foreigner_index",
            "conyuemp":"spouse_index","canal_entrada":"channel","cod_prov":"province","nomprov":"province_name",
            "ind_actividad_cliente":"active_index","renta":"income","segmento":"segment"}

train.rename(col_names, axis = 1, inplace = True)


In [7]:
# 2) Data Type Conversion
# Convert the features into their intuitive types
train.age = pd.to_numeric(train.age, errors='coerce')
train.income = pd.to_numeric(train.income, errors='coerce')
train.seniority = pd.to_numeric(train.seniority, errors='coerce')
train.first_date = pd.to_datetime(train.first_date, errors = 'coerce')
train['fecha_dato'] = pd.to_datetime(train['fecha_dato'])



In [8]:
# Drop the last primary date and spouse index fields given over 99% missing values
train.drop(['last_primary_date','spouse_index'], axis = 1, inplace = True)


In [9]:
train.isna().mean()


fecha_dato           0.000000
cust_id              0.000000
emp_index            0.002032
residence            0.002032
sex                  0.002037
age                  0.002032
first_date           0.002032
new_cust             0.002032
seniority            0.002032
is_primary           0.002032
cust_type            0.010975
cust_rel_type        0.010975
residence_index      0.002032
foreigner_index      0.002032
channel              0.013638
indfall              0.002032
tipodom              0.002032
province             0.006858
province_name        0.006858
active_index         0.002032
income               0.204756
segment              0.013876
ind_ahor_fin_ult1    0.000000
ind_aval_fin_ult1    0.000000
ind_cco_fin_ult1     0.000000
ind_cder_fin_ult1    0.000000
ind_cno_fin_ult1     0.000000
ind_ctju_fin_ult1    0.000000
ind_ctma_fin_ult1    0.000000
ind_ctop_fin_ult1    0.000000
ind_ctpp_fin_ult1    0.000000
ind_deco_fin_ult1    0.000000
ind_deme_fin_ult1    0.000000
ind_dela_f

# Version 1: No EDA

In [10]:
# 📌 Version 1: No EDA, Delete All Null Rows Except Income, Impute Income by Median (Grouped by Residence)

# ✅ Step 1: Copy train dataset into v1
v1 = train.copy()

# ✅ Step 2: Delete all rows having any null values, except 'income'
v1 = v1.dropna(subset=[col for col in v1.columns if col != 'income'])

# ✅ Step 3: Impute 'income' using median, grouped by 'residence'
v1['income'] = v1.groupby('province')['income'].transform(lambda x: x.fillna(x.median()))

# 🔍 Check result
print(v1.isna().sum())  # Should show no NaNs


fecha_dato           0
cust_id              0
emp_index            0
residence            0
sex                  0
age                  0
first_date           0
new_cust             0
seniority            0
is_primary           0
cust_type            0
cust_rel_type        0
residence_index      0
foreigner_index      0
channel              0
indfall              0
tipodom              0
province             0
province_name        0
active_index         0
income               0
segment              0
ind_ahor_fin_ult1    0
ind_aval_fin_ult1    0
ind_cco_fin_ult1     0
ind_cder_fin_ult1    0
ind_cno_fin_ult1     0
ind_ctju_fin_ult1    0
ind_ctma_fin_ult1    0
ind_ctop_fin_ult1    0
ind_ctpp_fin_ult1    0
ind_deco_fin_ult1    0
ind_deme_fin_ult1    0
ind_dela_fin_ult1    0
ind_ecue_fin_ult1    0
ind_fond_fin_ult1    0
ind_hip_fin_ult1     0
ind_plan_fin_ult1    0
ind_pres_fin_ult1    0
ind_reca_fin_ult1    0
ind_tjcr_fin_ult1    0
ind_valo_fin_ult1    0
ind_viv_fin_ult1     0
ind_nomina_

In [11]:
v1.head()

,fecha_dato,cust_id,emp_index,residence,sex,age,first_date,new_cust,seniority,is_primary,cust_type,cust_rel_type,residence_index,foreigner_index,channel,indfall,tipodom,province,province_name,active_index,income,segment,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
0,2015-01-28,1375586,N,ES,H,35.0,2015-01-12,0.0,6.0,1.0,1.0,A,S,N,KHL,N,1.0,29.0,MALAGA,1.0,87218.10,02 - PARTICULARES,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
1,2015-01-28,1050611,N,ES,V,23.0,2012-08-10,0.0,35.0,1.0,1.0,I,S,S,KHE,N,1.0,13.0,CIUDAD REAL,0.0,35548.74,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
2,2015-01-28,1050612,N,ES,V,23.0,2012-08-10,0.0,35.0,1.0,1.0,I,S,N,KHE,N,1.0,13.0,CIUDAD REAL,0.0,122179.11,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
3,2015-01-28,1050613,N,ES,H,22.0,2012-08-10,0.0,35.0,1.0,1.0,I,S,N,KHD,N,1.0,50.0,ZARAGOZA,0.0,119775.54,03 - UNIVERSITARIO,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
4,2015-01-28,1050614,N,ES,V,23.0,2012-08-10,0.0,35.0,1.0,1.0,A,S,N,KHE,N,1.0,50.0,ZARAGOZA,1.0,99992.49,03 - UNIVERSITARIO,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


# Function SPLIT for Train - Evaluation

In [12]:

def split_dataset(dataset):
    # Step 1: Filter data to include only January - May 2016
    cf_v1 = dataset[dataset['fecha_dato'].dt.year == 2016]

    # Step 2: Split into train (before 28/04/2016) and validation (only 28/04/2016)
    train_set = cf_v1[cf_v1['fecha_dato'] < '2016-04-28']
    eval_set = cf_v1[cf_v1['fecha_dato'] == '2016-04-28']

    # Return the train and validation sets
    return train_set, eval_set


## perform on v1

In [13]:
v1_train, v1_eval =split_dataset(v1)
v1_train.shape
v1_eval.head()


,fecha_dato,cust_id,emp_index,residence,sex,age,first_date,new_cust,seniority,is_primary,cust_type,cust_rel_type,residence_index,foreigner_index,channel,indfall,tipodom,province,province_name,active_index,income,segment,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
11787582,2016-04-28,896836,N,ES,H,72.0,2010-11-12,0.0,65.0,1.0,1.0,I,S,N,KFF,N,1.0,39.0,CANTABRIA,0.0,315824.52,02 - PARTICULARES,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
11787583,2016-04-28,896837,N,ES,H,57.0,2010-11-12,0.0,65.0,1.0,1.0,I,S,N,KAT,N,1.0,48.0,BIZKAIA,0.0,99978.78,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
11787584,2016-04-28,896839,N,ES,V,39.0,2010-11-12,0.0,65.0,1.0,1.0,A,S,N,KAZ,N,1.0,45.0,TOLEDO,1.0,46618.47,02 - PARTICULARES,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
11787585,2016-04-28,896853,N,ES,H,71.0,2010-11-12,0.0,65.0,1.0,1.0,I,S,N,KAT,N,1.0,50.0,ZARAGOZA,0.0,205204.11,02 - PARTICULARES,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
11787586,2016-04-28,896849,N,ES,V,25.0,2010-11-12,0.0,65.0,1.0,1.0,A,S,N,KFC,N,1.0,28.0,MADRID,1.0,100300.17,03 - UNIVERSITARIO,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0.0,0.0,0


# Funtion transform table into user-item matrix


In [14]:

def user_item_maker(train_set):
    """
    Creates a user-item matrix from the training set.

    Parameters:
    - train_set: The training dataset containing user interactions with products.

    Returns:
    - A DataFrame representing the user-item matrix.
    """
    # Select relevant columns for the user-item matrix
    train_set = train_set.iloc[:, [0, 1, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45]]

    # Create the user-item matrix
    user_item_matrix = train_set.groupby("cust_id").sum()
    user_item_matrix = user_item_matrix.fillna(0)
    user_item_matrix.index = user_item_matrix.index.astype('string')

    return user_item_matrix


In [15]:
user_item=user_item_maker(v1_train)
user_item.head()



,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,ind_cder_fin_ult1,ind_cno_fin_ult1,ind_ctju_fin_ult1,ind_ctma_fin_ult1,ind_ctop_fin_ult1,ind_ctpp_fin_ult1,ind_deco_fin_ult1,ind_deme_fin_ult1,ind_dela_fin_ult1,ind_ecue_fin_ult1,ind_fond_fin_ult1,ind_hip_fin_ult1,ind_plan_fin_ult1,ind_pres_fin_ult1,ind_reca_fin_ult1,ind_tjcr_fin_ult1,ind_valo_fin_ult1,ind_viv_fin_ult1,ind_nomina_ult1,ind_nom_pens_ult1,ind_recibo_ult1
cust_id,,,,,,,,,,,,,,,,,,,,,,,,
15889,0,0,3,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,2,3,0,0.0,0.0,0
15890,0,0,0,0,3,0,0,0,3,0,0,0,3,0,0,3,0,0,3,0,0,3.0,3.0,3
15892,0,0,3,0,0,0,0,0,0,0,0,3,3,0,0,0,0,3,3,3,0,0.0,0.0,3
15893,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0.0,0.0,0
15894,0,0,3,0,0,0,0,0,0,0,0,0,3,0,0,0,0,3,3,3,0,3.0,3.0,3


### Now let's find similarites between user

In [16]:
def calculate_user_similarity(user_item_matrix, top_n_users=10000):
    """
    Calculates the user similarity matrix using cosine similarity.

    Parameters:
    - user_item_matrix: The user-item matrix.
    - top_n_users: The number of top users to consider for similarity calculation.

    Returns:
    - A DataFrame representing the user similarity matrix.
    """
    # Select the top N users for similarity calculation
    df_cf_user = user_item_matrix[:top_n_users].copy()

    # Calculate cosine similarity for users
    similarity_matrix = pd.DataFrame(
        cosine_similarity(df_cf_user),
        index=df_cf_user.index.astype('string'),
        columns=df_cf_user.index.astype('string')
    )

    return similarity_matrix

In [17]:
user_sim=calculate_user_similarity(user_item, top_n_users=10000)
user_sim.head()

cust_id,15889,15890,15892,15893,15894,15895,15896,15897,15898,15899,15900,15901,15902,15903,15906,15907,15908,15910,15911,15913,15914,15915,15916,15917,15918,15919,15920,15921,15922,15923,15924,15925,15926,15927,15928,15929,15930,15932,15933,15934,15935,15936,15937,15938,15939,15940,15941,15942,15943,15944,15945,15947,15948,15949,15950,15951,15952,15954,15956,15957,15958,15959,15960,15961,15962,15964,15965,15967,15969,15972,15973,15974,15975,15976,15977,15978,15979,15980,15981,15982,15983,15984,15985,15986,15987,15988,15989,15990,15991,15992,15993,15994,15995,15996,15997,15998,15999,16000,16001,16002,16003,16004,16005,16007,16008,16009,16010,16011,16013,16017,16018,16019,16022,16024,16025,16026,16027,16028,16029,16031,16033,16034,16036,16038,16039,16040,16041,16042,16043,16046,16047,16048,16049,16050,16051,16052,16053,16055,16056,16060,16062,16063,16064,16065,16067,16070,16073,16074,16075,16082,16083,16085,16088,16090,16093,16094,16095,16098,16099,16100,16101,16102,16103,16105,16110,16111,16113,16114,16115,16117,16119,16120,16121,16122,16125,16126,16128,16131,16132,16133,16134,16135,16136,16137,16138,16140,16141,16145,16147,16148,16149,16152,16153,16155,16158,16161,16162,16163,16166,16167,16169,16170,16171,16174,16176,16177,16179,16181,16182,16183,16185,16187,16189,16192,16193,16194,16195,16196,16197,16198,16199,16201,16202,16203,16204,16205,16206,16207,16208,16209,16211,16213,16216,16217,16222,16227,16229,16230,16231,16232,16233,16234,16235,16236,16238,16240,16241,16242,16243,16244,16245,16246,16247,16249,16253,16255,16256,16257,16258,16259,16261,16263,16265,16266,16267,16268,16269,16270,16271,16273,16274,16276,16278,16279,16280,16281,16282,16283,16287,16289,16290,16291,16294,16296,16297,16306,16307,16308,16311,16324,16327,16329,16330,16338,16339,16341,16355,16357,16359,16360,16361,16362,16364,16365,16368,16404,16405,16411,16412,16418,16421,16424,16426,16445,16450,16456,16463,16465,16468,16478,16483,16484,16489,16490,16491,16494,16496,16497,16498,16499,16500,16502,16504,16506,16507,16508,16509,16510,16511,16512,16514,16515,16516,16517,16518,16521,16522,16525,16529,16530,16531,16532,16533,16534,16535,16536,16537,16538,16539,16540,16543,16544,16545,16546,16555,16556,16558,16560,16562,16563,16565,16568,16569,16571,16572,16574,16575,16576,16578,16579,16580,16583,16584,16585,16587,16590,16591,16592,16594,16595,16601,16603,16604,16607,16608,16610,16612,16613,16616,16617,16618,16620,16622,16629,16630,16631,16632,16633,16634,16635,16637,16638,16640,16641,16642,16647,16648,16649,16650,16652,16653,16656,16658,16660,16661,16662,16663,16666,16668,16671,16672,16673,16674,16675,16677,16680,16681,16685,16687,16690,16694,16695,16696,16699,16700,16702,16705,16707,16708,16709,16710,16711,16713,16714,16717,16719,16720,16721,16722,16723,16724,16725,16728,16730,16731,16735,16737,16738,16739,16741,16743,16746,16747,16749,16750,16751,16752,16757,16758,16759,16760,16761,16764,16765,16767,16768,16770,16771,16772,16773,16776,16777,16783,16785,16787,16788,16789,16791,16794,16795,16797,16801,16806,16807,16808,16813,16814,16820,16821,16824,16825,16826,16827,16828,16829,16830,16832,16834,16835,16836,16838,16839,16841,16842,16843,16844,16846,16848,16849,16850,16853,16854,16857,16861,16863,16866,16867,16868,16869,16871,16873,16877,16880,16882,16883,16886,16889,16890,16892,16894,16895,16898,16899,16901,16903,16905,16906,16908,16909,16910,16912,16914,16917,16919,16925,16926,16927,16930,16931,16934,16935,16936,16938,16940,16943,16944,16946,16952,16953,16954,16956,16958,16959,16962,16970,16972,16973,16974,16975,16976,16977,16978,16979,16981,16983,16984,16985,16986,16987,16988,16990,16992,16995,16998,16999,17001,17004,17006,17007,17008,17010,17011,17012,17013,17014,17015,17016,17017,17022,17026,17027,17029,17031,17032,17034,17035,17039,17041,17042,17043,17044,17045,17046,17047,17056,17058,17059,17062,17063,17064,17065,17067,17069,17071,17072,17074,17075,17076,17077,17079,17080,17085,17086,17088,17091,17094,17095,17098,17099,17100,17101,17103,17105,17106,17109,17

# Function User-base recommendation


In [18]:
def user_based_recommender(cust_id, sim_rate, top_n, similarity_matrix, user_item_matrix):
    """
    User-based recommendation model.

    Parameters:
    - cust_id: The customer ID for whom to recommend products.
    - sim_rate: Minimum similarity rate for other customers to be included.
    - top_n: The number of top products to recommend.
    - similarity_matrix: The cosine similarity matrix of users.
    - user_item_matrix: The user-item matrix containing product interactions.

    Returns:
    - A Series of top-N recommended products with predicted purchasing volumes.
    """
    cust_id = str(cust_id)

    # Find similar users with a similarity score above the threshold
    sim_cust = similarity_matrix.loc[
        (similarity_matrix[cust_id] >= sim_rate) & (similarity_matrix[cust_id] < 1), cust_id
    ]

    # Get products interacted by the target user
    products = user_item_matrix.loc[cust_id, :]

    # Find unpurchased products (assuming 0 = not purchased)
    unpurchased_products = products[products == 0].index

    # Create a new matrix with similar users and unpurchased products
    new_matrix = user_item_matrix.loc[sim_cust.index, unpurchased_products].copy()

    # Add similarity scores to the matrix
    new_matrix['sim_score'] = sim_cust.loc[new_matrix.index]

    # Apply similarity weight to each product
    for product in unpurchased_products:
        new_matrix[product] *= new_matrix['sim_score']

    # Calculate the predicted purchasing amount based on weighted sum
    top_item = new_matrix.drop(columns=['sim_score']).sum() / new_matrix['sim_score'].sum()

    # Sort to get top-N recommendations
    top_item = top_item.sort_values(ascending=False)[:top_n]

    # Filter out products with 0 predicted interest
    top_item = top_item[top_item > 0]

    # Show message if no recommendations are found
    if top_item.empty:
        print("There is no recommendation for the customer")

    return top_item

# Example usage:
# Assuming `similarity_matrix` and `user_item_matrix` are already defined
# recommendations = user_based_recommender(cust_id, sim_rate, top_n, similarity_matrix, user_item_matrix)


In [19]:
user_based_recommender(cust_id = 15929, sim_rate = 0.70, top_n = 10, similarity_matrix=user_sim, user_item_matrix= user_item)

ind_reca_fin_ult1    0.766523
ind_nom_pens_ult1    0.564757
ind_ctop_fin_ult1    0.489762
ind_nomina_ult1      0.437036
ind_fond_fin_ult1    0.407200
ind_plan_fin_ult1    0.344046
ind_dela_fin_ult1    0.325189
ind_cno_fin_ult1     0.300629
ind_hip_fin_ult1     0.054660
ind_deco_fin_ult1    0.008728
dtype: float64

In [20]:
# Testing Case 2: cust_id "16117", similarity score min 0.80, top 5 products
user_based_recommender(cust_id = 16117, sim_rate = 0.70, top_n = 10, similarity_matrix=user_sim, user_item_matrix= user_item)

ind_recibo_ult1      0.685851
ind_ecue_fin_ult1    0.300689
ind_ctop_fin_ult1    0.271374
ind_fond_fin_ult1    0.157860
ind_ctpp_fin_ult1    0.141445
ind_reca_fin_ult1    0.106589
ind_dela_fin_ult1    0.055640
ind_plan_fin_ult1    0.030670
ind_nom_pens_ult1    0.030519
ind_nomina_ult1      0.020217
dtype: float64

In [21]:
# Testing Case 3: cust_id "16117", similarity score min 0.80, top 5 products


From the recommendations above, we can find for each customer, the model will recommend top N products the customer haven't purchased. We can found that most of the product recommended to the customers are having very low predicted purchasing volume (weighted average volume). This is one of the big challenges of collaborative filtering which is **Sparsity**, meaning that there are many missing values (zeros) in the the user-item matrix due to **the users have not purchased or interacted with certain items.** This sparsity can **make it** **challenging to accurately predict scores or recommendations for those items.**

Besides, in real life scienrios, as the customer population usually grows with rapid speed, it is costly to maintain the storage and computation of the user-item matrix and user similarity matrix. 

Therefore, the user-based recommender is not working well under this scienrio.

#### 2.2) Item-based Collaborative Filtering

Item-based collaborative filtering is another approach used in recommender systems that **focuses on similarities between items rather than similarities between users.** In item-based collaborative filtering, recommendations are made based on the **similarities between items**, and users' preferences for items are used to determine the recommendations.



# Funtion item-based recommendation


### Similarities between Items

In [22]:
def calculate_item_similarity(user_item_matrix, top_n_users=10000):
    """
# 1. Based on the User-Item Matrix above, use Cosine Similarity to calculate the item similarities and build the similarity matrix
# Given the memory limitation (RAM), only using the top 10,000 customers' into to build the model. 
df_cf_item = df_cf.copy()
df_cf_item = df_cf_item[:10000].T

df_cf_item = pd.DataFrame(cosine_similarity(df_cf_item), index = df_cf_item.index.astype('string'), columns = df_cf_item.index.astype('string'))
df_cf_item.shape
    """
    # Select the top N users and transpose the matrix for item similarity calculation    df_cf_item = user_item_matrix.copy()
    df_cf_item = user_item_matrix[:top_n_users].T
    # Calculate cosine similarity for items
    similarity_matrix = pd.DataFrame(
        cosine_similarity(df_cf_item),
        index=df_cf_item.index.astype('string'),
        columns=df_cf_item.index.astype('string')
    )

    return similarity_matrix

In [23]:
item_sim = calculate_item_similarity(user_item, top_n_users=10000)
item_sim.shape

(24, 24)

In [24]:
def item_based_recommender(cust_id, sim_rate, top_n, similarity_matrix, user_item_matrix):
    """
    Item-based recommendation model.

    Parameters:
    - cust_id: The customer ID for whom to recommend products.
    - sim_rate: Minimum similarity rate for other items to be included.
    - top_n: The number of top products to recommend.
    - item_similarity_matrix: The cosine similarity matrix of items.
    - user_item_matrix: The user-item matrix containing product interactions.

    Returns:
    - A Series of top-N recommended products with predicted purchasing volumes.
    """
    cust_id = str(cust_id)

    # Get the products the target user has interacted with
    user_products = user_item_matrix.loc[cust_id, :]

    # Find purchased products (assuming non-zero values indicate purchased products)
    purchased_products = user_products[user_products > 0].index

    # Check if the customer has purchased any product
    if purchased_products.empty:
        return pd.Series(dtype=float)  # Return an empty Series

    # Initialize a dictionary to store the predicted purchasing volumes
    predicted_volumes = {}

    # Loop through each purchased product to find similar items
    for product in purchased_products:
        # Get similar items with a similarity score above the threshold
        sim_items = similarity_matrix.loc[
            (similarity_matrix[product] >= sim_rate) & (similarity_matrix[product] < 1), product
        ]

        # Calculate the predicted purchasing volume for each similar item
        for sim_item, sim_score in sim_items.items():
            if sim_item not in purchased_products:  # Exclude already purchased items
                if sim_item in predicted_volumes:
                    predicted_volumes[sim_item] += sim_score * user_products[product]
                else:
                    predicted_volumes[sim_item] = sim_score * user_products[product]

    # Convert the predicted volumes to a Series and sort to get top-N recommendations
    top_item = pd.Series(predicted_volumes).sort_values(ascending=False)[:top_n]

    # Filter out products with 0 predicted interest
    top_item = top_item[top_item > 0]

    # Show message if no recommendations are found
    if top_item.empty:
        print("There is no recommendation for the customer")

    return top_item

# Example usage:
# Assuming `item_similarity_matrix` and `user_item_matrix` are already defined
# recommendations = item_based_recommender(cust_id, sim_rate, top_n, item_similarity_matrix, user_item_matrix)


In [25]:
user_item['ind_cco_fin_ult1']

cust_id
15889      3
15890      0
15892      3
15893      0
15894      3
          ..
1536368    0
1536369    1
1536371    0
1536372    1
1536373    1
Name: ind_cco_fin_ult1, Length: 915947, dtype: int64

In [26]:


item_based_recommender(cust_id = "16117", sim_rate = 0.0, top_n = 10, similarity_matrix=item_sim, user_item_matrix= user_item)


ind_recibo_ult1      4.222371
ind_ecue_fin_ult1    3.482953
ind_nom_pens_ult1    3.088315
ind_reca_fin_ult1    2.956647
ind_cno_fin_ult1     2.871335
ind_nomina_ult1      2.839431
ind_ctop_fin_ult1    2.663585
ind_fond_fin_ult1    2.559962
ind_ctpp_fin_ult1    2.283880
ind_plan_fin_ult1    2.114262
dtype: float64

# Funtion for prediction

In [27]:
import pandas as pd

def generate_predictions(recommender_function, user_item_matrix, similarity_matrix, sim_rate=0.7, top_n=1, num_users=1000):
    """
    Generates predictions using a specified recommender function.

    Parameters:
    - recommender_function: The recommender function to use for predictions.
    - user_item_matrix: The user-item matrix containing user interactions with products.
    - similarity_matrix: The similarity matrix (item-based or user-based) used by the recommender function.
    - sim_rate: Minimum similarity rate for recommendations.
    - top_n: Number of top recommendations to generate.
    - num_users: Number of users to generate predictions for.

    Returns:
    - A DataFrame with user IDs and their predicted items.
    """
    # Initialize a list to store predictions
    predictions = []

    # Loop through the specified number of users in the user-item matrix
    for user in user_item_matrix.index[:num_users]:
        # Generate prediction using the specified recommender function
        prediction = recommender_function(
            cust_id=user,
            sim_rate=sim_rate,
            top_n=top_n,
            similarity_matrix=similarity_matrix,
            user_item_matrix=user_item_matrix
        )
        predictions.append((user, prediction))  # Store (user_id, predicted_item)

    # Convert to DataFrame for better analysis
    df_predictions = pd.DataFrame(predictions, columns=["user_id", "predicted_item"])

    return df_predictions


In [28]:
generate_predictions(item_based_recommender, user_item, sim_rate=0, top_n=1, num_users=1000,similarity_matrix=item_sim)


,user_id,predicted_item
0,15889,ind_recibo_ult1 4.912058 dtype: float64
1,15890,ind_reca_fin_ult1 10.130231 dtype: float64
2,15892,ind_nom_pens_ult1 8.935138 dtype: float64
3,15893,ind_recibo_ult1 1.245712 dtype: float64
4,15894,ind_cno_fin_ult1 12.929156 dtype: float64
...,...,...
995,17713,ind_recibo_ult1 3.779138 dtype: float64
996,17714,ind_cno_fin_ult1 4.837143 dtype: float64
997,17716,ind_nom_pens_ult1 6.536867 dtype: float64
998,17718,ind_ctop_fin_ult1 1.663951 dtype: float64


# Funtion Calculate Map

In [29]:
def calculate_map(merged_df):
    total_ap = 0
    num_users = len(merged_df)

    for _, row in merged_df.iterrows():
        actual_purchases = row['products_purchased']
        predicted_item = row['predicted_item']
        precision_at_1 = 1 if predicted_item in actual_purchases else 0
        total_ap += precision_at_1

    return total_ap / num_users if num_users > 0 else 0




## calculate predictions and actual products 

In [30]:
df_predictions=generate_predictions(item_based_recommender, user_item, sim_rate=0, top_n=1, num_users=1000,similarity_matrix=item_sim)

# Melt the DataFrame to long format
product_columns = [col for col in v1_eval.columns if col.startswith('ind_')]
df_melted = v1_eval.melt(id_vars=['cust_id'], value_vars=product_columns, var_name='product', value_name='purchased')

# Filter rows where the product was purchased
df_purchased = df_melted[df_melted['purchased'] == 1]

# Group by cust_id and aggregate product names into a list
df_actual = df_purchased.groupby('cust_id')['product'].apply(list).reset_index()

# Rename columns to match the desired format
df_actual.columns = ['cust_id', 'products_purchased']

# Display the resulting DataFrame
print(df_predictions.head())

  user_id                                 predicted_item
0   15889     ind_recibo_ult1    4.912058
dtype: float64
1   15890  ind_reca_fin_ult1    10.130231
dtype: float64
2   15892   ind_nom_pens_ult1    8.935138
dtype: float64
3   15893     ind_recibo_ult1    1.245712
dtype: float64
4   15894   ind_cno_fin_ult1    12.929156
dtype: float64


## Create table including columns of prediction and actual bought products


In [31]:
# Make sure that 2 tables are similar before merging
df_predictions['predicted_item'] = df_predictions['predicted_item'].astype(str)
df_predictions['predicted_item'] = df_predictions['predicted_item'].str.split().str[0]
df_predictions.head()
df_predictions['user_id'] = df_predictions['user_id'].astype(int)
df_actual['cust_id'] = df_actual['cust_id'].astype(int)
merged_df = df_predictions.merge(df_actual, left_on='user_id', right_on='cust_id', how='inner')



### Let's finger crossed

In [32]:

map_score = calculate_map(merged_df)
print(f"MAP Score: {map_score:.4f}")

MAP Score: 0.0141
